In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [18]:
phi_eval_c = pd.DataFrame(json.loads(open("./phi2_eval_df_c.json", "r").read()))
phi_eval_h = pd.DataFrame(json.loads(open("./phi2_eval_hard.json", "r").read()))
def_eval_c = pd.DataFrame(json.loads(open("./sqlc_eval_df_c.json", "r").read()))
def_eval_h = pd.DataFrame(json.loads(open("./sqlc_eval_hard.json", "r").read()))

phi_eval_c.shape, phi_eval_h.shape, def_eval_c.shape, def_eval_h.shape 

((1258, 9), (33, 9), (1258, 9), (33, 9))

In [21]:
phi_eval = pd.concat([phi_eval_c, phi_eval_h])
def_eval = pd.concat([def_eval_c, def_eval_h])

phi_eval.shape, def_eval.shape

((1291, 9), (1291, 9))

In [22]:
# Plot between time and difficulty
phi_eval.groupby('difficulty')['inf_time'].mean().reset_index()

,difficulty,inf_time
0,easy,16.261769
1,hard,38.548714
2,medium,102.065706


In [24]:
def_eval.sort_values(by='inf_time', ascending=False)

,prompt,pred,actu,inf_time,temperature,difficulty,token_in,token_out,tokens_per_sec
0,### Task\nGenerate a SQL query to answer the f...,"SELECT MAX(g.gradepoint) AS max_grade_point, M...","SELECT MAX(T2.gradepoint), MIN(T2.gradepoint) ...",468.298223,0.2,hard,145,64,0.134530
17,### Task\nGenerate a SQL query to answer the f...,"SELECT t.task_details, t.task_id, t.project_id...","SELECT T1.task_details, T1.task_id, T2.project...",66.946423,0.2,hard,174,112,1.658042
9,### Task\nGenerate a SQL query to answer the f...,"SELECT t.month, t._year FROM table_name_24 t W...","SELECT month, _year FROM table_name_24 WHERE g...",64.969241,0.2,medium,259,51,0.769595
28,### Task\nGenerate a SQL query to answer the f...,SELECT ta.name FROM Tourist_Attractions ta JOI...,SELECT T1.Name FROM Tourist_Attractions AS T1 ...,64.069760,0.2,hard,169,107,1.654447
640,### Task\nGenerate a SQL query to answer the f...,SELECT t.location FROM table_name_77 t WHERE t...,SELECT location FROM table_name_77 WHERE faste...,62.517787,0.2,medium,174,51,0.799772
...,...,...,...,...,...,...,...,...,...
619,### Task\nGenerate a SQL query to answer the f...,SELECT m.country FROM manager m GROUP BY m.cou...,SELECT DISTINCT Country FROM manager,21.227452,0.2,easy,85,23,1.036394
369,### Task\nGenerate a SQL query to answer the f...,SELECT COUNT(DISTINCT c.id) FROM company c WHE...,SELECT COUNT(*) FROM company,21.208672,0.2,easy,84,24,1.084462
407,### Task\nGenerate a SQL query to answer the f...,SELECT COUNT(c.id) AS number_of_candidates FRO...,SELECT COUNT(*) FROM candidate,19.343117,0.2,easy,82,19,0.930564
1013,### Task\nGenerate a SQL query to answer the f...,SELECT COUNT(s.id) AS number_of_songs FROM son...,SELECT COUNT(*) FROM Songs,18.762099,0.2,easy,82,18,0.906082


In [ ]:
# token_in vs inf_time - scatter
# difficulty vs inf_time - bar
# training_logs - line
# 